# Timesfm Monthly

In [1]:
!pip install timesfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of googleapis-common-protos to determine which version is compatible with other requirements. This could take a while.
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.2/440.2 kB 21.2 MB/s eta

In [1]:
import pandas as pd
import timesfm

2024-09-12 05:38:51.680118: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64


In [2]:
data = pd.read_excel("monthly_data.xlsx")

In [3]:
data.head()

,date,warehouse,orders,holiday_name,holiday,shutdown,mini_shutdown,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow,user_activity_1,user_activity_2,id
0,2020-12-01,Prague_1,190208,Christmas Eve2nd Christmas Day,2,0,0,1,0,0,0,0,0,8.3,0.0,45470,839751,Prague_1_2020-12-05Prague_1_2020-12-06Prague_1...
1,2021-01-01,Prague_1,235089,New Years Day,1,0,0,1,0,0,0,0,0,166.3,112.0,53790,996212,Prague_1_2021-01-01Prague_1_2021-01-02Prague_1...
2,2021-02-01,Prague_1,222102,0,0,0,0,0,0,0,0,0,0,37.0,134.0,49832,943849,Prague_1_2021-02-01Prague_1_2021-02-02Prague_1...
3,2021-03-01,Prague_1,254258,International womens day,0,0,0,0,14,0,0,0,0,24.9,3.0,55508,1094886,Prague_1_2021-03-01Prague_1_2021-03-02Prague_1...
4,2021-04-01,Prague_1,242598,Good FridayEaster Monday,4,0,0,1,0,0,0,0,0,13.4,2.0,50988,1041067,Prague_1_2021-04-01Prague_1_2021-04-02Prague_1...


In [4]:
data = data.drop(["holiday_name", 'shutdown', 'mini_shutdown', 'blackout', 'frankfurt_shutdown', "id", 'blackout', 'mov_change', 'frankfurt_shutdown','precipitation', 'snow', 'user_activity_1', 'user_activity_2'], axis=1)

In [5]:
#rename as per requirement of the forecast libraries
data = data.rename(columns={"warehouse": "unique_id", "date": "ds", "orders": "y"})

In [6]:
data.head()

,ds,unique_id,y,holiday,shops_closed,winter_school_holidays,school_holidays
0,2020-12-01,Prague_1,190208,2,1,0,0
1,2021-01-01,Prague_1,235089,1,1,0,0
2,2021-02-01,Prague_1,222102,0,0,0,0
3,2021-03-01,Prague_1,254258,0,0,14,0
4,2021-04-01,Prague_1,242598,4,1,0,0


In [7]:
data['ds'] = pd.to_datetime(data['ds'])

In [9]:
#Splitting the Data for Trasin & Test
train = data[data['ds'] <= '2023-11-01']
test = data[data['ds'] > '2023-11-01']

In [10]:
Y_test_df = test.copy()
df = train.copy()

In [16]:
tfm = timesfm.TimesFm(
    context_len=512,
    horizon_len=4,
    input_patch_len=32,
    output_patch_len=128,
    num_layers=20,
    model_dims=1280,
    backend="cpu",
)
tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")

Multiprocessing context has already been set.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Constructing model weights.


Constructed model weights in 4.79 seconds.
Restoring checkpoint from /root/.cache/huggingface/hub/models--google--timesfm-1.0-200m/snapshots/8775f7531211ac864b739fe776b0b255c277e2be/checkpoints.


ERROR:absl:For checkpoint version > 1.0, we require users to provide
          `train_state_unpadded_shape_dtype_struct` during checkpoint
          saving/restoring, to avoid potential silent bugs when loading
          checkpoints to incompatible unpadded shapes of TrainState.


Restored checkpoint in 6.24 seconds.
Jitting decoding.
Jitted decoding in 36.04 seconds.


In [17]:
timesfm_df = tfm.forecast_on_df(
    inputs=df,
    freq='MS',
    value_name='y',
    num_jobs=-1,
)

Processing dataframe with multiple processes.


2024-09-12 05:43:04.386602: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-09-12 05:43:04.386602: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64


Finished preprocessing dataframe.
Finished forecasting.


In [18]:
timesfm_df

,unique_id,ds,timesfm,timesfm-q-0.1,timesfm-q-0.2,timesfm-q-0.3,timesfm-q-0.4,timesfm-q-0.5,timesfm-q-0.6,timesfm-q-0.7,timesfm-q-0.8,timesfm-q-0.9
0,Brno_1,2023-12-01,254967.671875,237663.187500,244006.734375,248769.546875,251503.109375,254967.671875,258723.906250,262036.562500,267336.281250,273261.406250
1,Brno_1,2024-01-01,249670.328125,229482.062500,236937.875000,240994.609375,245445.953125,249670.328125,252853.812500,257846.812500,262228.656250,271091.156250
2,Brno_1,2024-02-01,237515.640625,215260.578125,223674.343750,228517.796875,233477.484375,237515.640625,242161.828125,246734.125000,253021.515625,261704.968750
3,Brno_1,2024-03-01,240161.140625,215913.062500,224705.750000,230042.828125,236146.484375,240161.140625,245142.062500,249891.625000,256198.062500,265958.093750
4,Prague_1,2023-12-01,292951.656250,274931.562500,281578.468750,286305.718750,289015.562500,292951.656250,297177.281250,300200.406250,305806.125000,311400.656250
5,Prague_1,2024-01-01,286850.062500,265308.718750,272303.312500,277346.312500,282862.968750,286850.062500,290022.906250,294835.875000,299486.312500,308423.875000
6,Prague_1,2024-02-01,278267.812500,254396.062500,262350.062500,268459.687500,274067.906250,278267.812500,284109.875000,288447.031250,295159.656250,303410.968750
7,Prague_1,2024-03-01,283899.843750,256514.078125,266089.625000,271967.093750,279307.593750,283899.843750,288984.500000,294547.562500,301514.125000,310367.031250
8,Prague_2,2023-12-01,181069.250000,170029.703125,174036.468750,177163.171875,178865.390625,181069.250000,183799.953125,185816.140625,189236.328125,192830.421875
9,Prague_2,2024-01-01,172745.281250,159750.390625,164276.953125,166896.156250,170449.484375,172745.281250,174985.640625,177796.609375,180389.734375,186080.015625


In [19]:
timesfm_df.to_csv("timesfm_monthly_4.csv")